In [2]:
import pandas as pd
import ast

In [66]:
# Create a Pandas dataframe to act as our "table" for now

# Import csv file of scraped data from Class Roster
# Each row is a Cornell class
classes_df = pd.read_csv("./roster_api_data.csv", na_filter= False)

# Find all classes with Subject ENGRD and number 2110

subject = "ENGRD"
number = 2110

# Result is a table with one row

result = classes_df[(classes_df["subject"] == subject) & (classes_df["number"] == number)]

# Display that table
display(result)

# Since result is a table with one row
#   we need .iloc[0] to choose only that first row
#   then we get the "professors" column and "cast" it to a list
#   using ast.literal_eval
professor_list = ast.literal_eval(result.iloc[0]["professors"])

print(professor_list)



,id,subject,number,title,description,outcomes,professors
96,358546,ENGRD,2110,Object-Oriented Programming and Data Structures,Intermediate programming in a high-level langu...,['Be fluent in the use of recursion and object...,"['Eleanor Jane Birrell', 'Michael Ryan Clarkso..."


['Eleanor Jane Birrell', 'Michael Ryan Clarkson', 'David Joseph Gries', 'Anne  Bracy']


In [69]:
# Import csv file of scraped data from RateMyProfessor.com
ratemyprof_df = pd.read_csv("./ratemyprofessor_api_data.csv", na_filter= False)

# Get only the rows for professors that have reviews and tags
reviewed_professors = ratemyprof_df[ratemyprof_df["review"] != ""]

display(reviewed_professors)

,professor,review,tags
7,So-Yeon Yoon,"Lectures are crazy fast and confusing, and pro...","['Graded by few things', 'Group projects', 'Gi..."
9,Kristen Nancy Angierski,Ms. A is very nice and genuinely cares about h...,"['Gives good feedback', 'Inspirational', 'Cari..."
12,Vicki L. Bogan,Prof. Bogan is awesome. She knows her stuff an...,"['Participation matters', 'EXTRA CREDIT', 'Giv..."
15,Ernesto Quinonez,The fact that this is my first time giving a r...,"['Participation matters', 'Hilarious', 'Would ..."
22,Chris Fromme,I already miss Fromme. Probably my favorite pr...,"['Inspirational', 'Caring', 'Test heavy']"
...,...,...,...
1915,Saida Hodzic,I loved this class! Hodzic is a good professor...,"['Get ready to read', 'Participation matters',..."
1917,Robert Connelly,Sometimes has a tough time communicating exact...,"['Tough grader', 'Lecture heavy', 'Get ready t..."
1923,Andrew J. Hicks,"The most inspiring, knowledgeable professor I ...","[""Skip class? You won't pass."", 'Get ready to ..."
1925,Richard Herbert Rand,He was a great lecturer,"['Hilarious', 'Respected', 'Would take again',..."
